In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def check_3mon(aa):
    A0  = pd.read_csv(aa+".csv",encoding="utf-8-sig")
    
    A0["de_dt"] = pd.to_datetime(A0["de_dt"].astype(int).astype(str))

    A0_test = A0[A0["de_dt"] >= "2021-10-01"]
    A0 = A0[A0["de_dt"] < "2021-10-01"]


    #raitings : 고객 별 상품 구매 횟수 count (대분류는 덤으로!)
    A0_tmp = A0.value_counts(['cust','pd_c',"de_dt","clac_mcls_nm","clac_hlv_nm"]).reset_index().rename(columns={0:"buy_am"})
    ratings=A0_tmp.groupby(['cust','pd_c',"clac_mcls_nm","clac_hlv_nm"],as_index=False).count()[['cust','pd_c',"clac_mcls_nm","clac_hlv_nm",'buy_am']]

    #고객 별 상품 대분류 별 구매 횟수
    tmp = A0_tmp.groupby(['cust','clac_hlv_nm'],as_index=False).count()[['cust','clac_hlv_nm','buy_am']]

    #고객 별 구매 횟수
    tmp_ref = A0_tmp.groupby("cust",as_index=False).sum()[["cust","buy_am"]].rename(columns={"buy_am":"buy_sum"})

    #참조1 : 고객 별 대분류 구매 비율 생성

    ref1 = pd.merge(tmp,tmp_ref, how='left',on='cust')
    ref1["buy_score_ref"] = ref1["buy_am"]/ref1["buy_sum"]
    ref1 = ref1.drop(["buy_am","buy_sum"],axis=1)

    #참조2 : 고객 별 상품 별 마지막 구매 날짜
    score_update=A0_tmp.value_counts(['cust','pd_c','de_dt']).reset_index()[['cust','pd_c','de_dt']]
    ref2 = score_update.groupby(["cust","pd_c"],as_index=False).max("de_dt").rename(columns={"de_dt":"last_dt"})

    #참조3 : 상품 별 고객 마다의 구매 간격들 평균 

    tmp = A0_tmp.value_counts(['cust','pd_c','de_dt'])
    date_file = tmp.reset_index()[['cust','pd_c','de_dt']]

    qq = date_file.value_counts(["cust","pd_c"])[date_file.value_counts(["cust","pd_c"]) > 1].reset_index()[['cust','pd_c']]
    qq

    range_lst = []
    for n in range(len(qq)):
        i = qq.loc[n,"cust"]
        j = qq.loc[n,"pd_c"]
        tmp = date_file[(date_file["pd_c"] == j)&(date_file["cust"] == i)].sort_values("de_dt")
        for k in range(len(tmp)-1):
            range_lst.append({"pd_c":j,"range_mean":(tmp.iloc[k+1,2] - tmp.iloc[k,2]).days})
    range_lst = pd.DataFrame(range_lst)

    ref3 = range_lst.groupby("pd_c",as_index=False).mean("range_mean")[["pd_c","range_mean"]]

    #합쳐주기!
    ratings = pd.merge(ratings,ref1, how='left',on=['cust',"clac_hlv_nm"])
    ratings = pd.merge(ratings,ref2, how='left',on=['cust',"pd_c"])
    ratings = pd.merge(ratings,ref3, how='left',on='pd_c')
    ratings = ratings.fillna(0)


    #구매 횟수 0~5점
    ratings['buy_level']=np.where(ratings.buy_am>=24,5,
                           np.where(ratings.buy_am>=12,4,
                                   np.where(ratings.buy_am>=6,3,
                                           np.where(ratings.buy_am>=4,2,
                                                   np.where(ratings.buy_am>=3,1,0)))))

    ratings['buy_level']=ratings['buy_level'].astype('float')

    #대분류 0~5점

    low_clac_lst = A0_tmp.value_counts(["cust","clac_hlv_nm"]).reset_index().rename(columns={0:"count"}).groupby("clac_hlv_nm",as_index=False).mean("count")[["clac_hlv_nm","count"]]

    low_clac_lst["low_clac"] = np.where(low_clac_lst["count"] >= low_clac_lst["count"].quantile(q=15/21),5,
                           np.where(low_clac_lst["count"]>=low_clac_lst["count"].quantile(q=10/21),4,
                                   np.where(low_clac_lst["count"]>=low_clac_lst["count"].quantile(q=6/21),3,
                                           np.where(low_clac_lst["count"]>=low_clac_lst["count"].quantile(q=3/21),2,
                                                   np.where(low_clac_lst["count"]>=low_clac_lst["count"].quantile(q=1/21),1,0)))))


    ratings = pd.merge(ratings, low_clac_lst.drop("count",axis=1), how='left',on = 'clac_hlv_nm')

    #(구매 간격 / 평균 구매 간격)의 quantile 별 점수

    ##range_mean이 0인 애들의 대체값, 같은 대분류들 중 최대 구매간격
    tmp_value = ratings[ratings["range_mean"] == 0].value_counts("clac_hlv_nm").reset_index().drop(0,axis=1)
    tmp_m = []
    for i in tmp_value["clac_hlv_nm"]:
        tmp_m.append(ratings[(ratings["clac_hlv_nm"] == i) & (ratings["range_mean"] != 0)]["range_mean"].max())
    tmp_value["tmp_m"] = tmp_m

    ratings = pd.merge(ratings, tmp_value, how='left',on="clac_hlv_nm")
    ratings.loc[ratings["range_mean"] != 0,"tmp_m"] = 0
    ratings["tmp_m"] = ratings["tmp_m"].fillna(ratings["range_mean"].max())

    today = pd.datetime(year = 2022, month = 1, day = 1)
    ratings["range_result"] = (today - ratings["last_dt"]).dt.days / (ratings["range_mean"] + ratings["tmp_m"])


    ratings["range_score"] = np.where(ratings["range_result"] >= ratings["range_result"].quantile(q=15/21),5,
                           np.where(ratings["range_result"]>=ratings["range_result"].quantile(q=10/21),4,
                                   np.where(ratings["range_result"]>=ratings["range_result"].quantile(q=6/21),3,
                                           np.where(ratings["range_result"]>=ratings["range_result"].quantile(q=3/21),2,
                                                   np.where(ratings["range_result"]>=ratings["range_result"].quantile(q=1/21),1,0)))))
    
    
    ratings["score"] = (ratings["buy_level"]*2 + ratings["low_clac"] + ratings["range_score"])/4
    
    

    A0_test_tmp = A0_test.value_counts(['cust','pd_c',"de_dt","clac_mcls_nm","clac_hlv_nm"]).reset_index().rename(columns={0:"buy_am"})
    A0_test2 = A0_test_tmp.groupby(["cust","pd_c"],as_index=False).sum()[["cust","pd_c"]]
    A0_test2["aft_buy"] = 1


    rrr = pd.merge(ratings,A0_test2,how="left",on=["cust","pd_c"]).fillna(0)
    
    print("상위10% 점수의 10~12월 구매 비율")
    print(rrr.loc[rrr["score"] >= rrr["score"].quantile(0.9),"aft_buy"].mean())
    print("\n")
    
    for i in np.arange(0,5,0.5):
        print(f"{i}보다 큰 값들의 10~12월 구매 비율")
        print(rrr[rrr['score']>=i]["aft_buy"].mean(),"\n")
        
    return 0

# A01

In [3]:
check_3mon("A01")

<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


상위10% 점수의 10~12월 구매 비율
0.22533439557214138


0.0보다 큰 값들의 10~12월 구매 비율
0.16428816613747532 

0.5보다 큰 값들의 10~12월 구매 비율
0.16441264315950063 

1.0보다 큰 값들의 10~12월 구매 비율
0.16498113010661056 

1.5보다 큰 값들의 10~12월 구매 비율
0.16874504263916984 

2.0보다 큰 값들의 10~12월 구매 비율
0.17687579155469063 

2.5보다 큰 값들의 10~12월 구매 비율
0.22533439557214138 

3.0보다 큰 값들의 10~12월 구매 비율
0.558177781432448 

3.5보다 큰 값들의 10~12월 구매 비율
0.6306990881458967 

4.0보다 큰 값들의 10~12월 구매 비율
0.7003424657534246 

4.5보다 큰 값들의 10~12월 구매 비율
0.7326732673267327 



0

# A02

In [4]:
check_3mon("A02")

<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


상위10% 점수의 10~12월 구매 비율
0.45768169724815677


0.0보다 큰 값들의 10~12월 구매 비율
0.17543482110357989 

0.5보다 큰 값들의 10~12월 구매 비율
0.17544355885033186 

1.0보다 큰 값들의 10~12월 구매 비율
0.1760338476164069 

1.5보다 큰 값들의 10~12월 구매 비율
0.1801375038678773 

2.0보다 큰 값들의 10~12월 구매 비율
0.19050498218573972 

2.5보다 큰 값들의 10~12월 구매 비율
0.24411106590724166 

3.0보다 큰 값들의 10~12월 구매 비율
0.47926885346413245 

3.5보다 큰 값들의 10~12월 구매 비율
0.5216699097824165 

4.0보다 큰 값들의 10~12월 구매 비율
0.5362537764350453 

4.5보다 큰 값들의 10~12월 구매 비율
0.5674869500372856 



0

# A03

In [5]:
check_3mon("A03")

<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


상위10% 점수의 10~12월 구매 비율
0.5261952131187074


0.0보다 큰 값들의 10~12월 구매 비율
0.20061387875187417 

0.5보다 큰 값들의 10~12월 구매 비율
0.20069057255257125 

1.0보다 큰 값들의 10~12월 구매 비율
0.20113516564260386 

1.5보다 큰 값들의 10~12월 구매 비율
0.20506982563316464 

2.0보다 큰 값들의 10~12월 구매 비율
0.21381048995528978 

2.5보다 큰 값들의 10~12월 구매 비율
0.2681514014489997 

3.0보다 큰 값들의 10~12월 구매 비율
0.5261952131187074 

3.5보다 큰 값들의 10~12월 구매 비율
0.5938071596550824 

4.0보다 큰 값들의 10~12월 구매 비율
0.6729758149316509 

4.5보다 큰 값들의 10~12월 구매 비율
0.7702564102564102 



0

# A04

In [6]:
check_3mon("A04")

상위10% 점수의 10~12월 구매 비율
0.5012227656736328


0.0보다 큰 값들의 10~12월 구매 비율
0.1938904380228327 

0.5보다 큰 값들의 10~12월 구매 비율
0.19393893912673696 

1.0보다 큰 값들의 10~12월 구매 비율
0.19531358554916944 

1.5보다 큰 값들의 10~12월 구매 비율
0.1993364299933643 

2.0보다 큰 값들의 10~12월 구매 비율
0.2031679989516446 

2.5보다 큰 값들의 10~12월 구매 비율
0.2572606274188981 

3.0보다 큰 값들의 10~12월 구매 비율
0.5012227656736328 

3.5보다 큰 값들의 10~12월 구매 비율
0.5467779918646971 

4.0보다 큰 값들의 10~12월 구매 비율
0.604911906033102 

4.5보다 큰 값들의 10~12월 구매 비율
0.6262458471760798 



<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


0

# A05

In [7]:
check_3mon("A05")

상위10% 점수의 10~12월 구매 비율
0.01937046004842615


0.0보다 큰 값들의 10~12월 구매 비율
0.03140786294750714 

0.5보다 큰 값들의 10~12월 구매 비율
0.03140786294750714 

1.0보다 큰 값들의 10~12월 구매 비율
0.03148727110317106 

1.5보다 큰 값들의 10~12월 구매 비율
0.029754820280885504 

2.0보다 큰 값들의 10~12월 구매 비율
0.027666666666666666 

2.5보다 큰 값들의 10~12월 구매 비율
0.01937046004842615 

3.0보다 큰 값들의 10~12월 구매 비율
0.4 

3.5보다 큰 값들의 10~12월 구매 비율
0.375 

4.0보다 큰 값들의 10~12월 구매 비율
1.0 

4.5보다 큰 값들의 10~12월 구매 비율
nan 



<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


0

# A06

In [8]:
check_3mon("A06")

상위10% 점수의 10~12월 구매 비율
0.04968728283530229


0.0보다 큰 값들의 10~12월 구매 비율
0.023969433039846215 

0.5보다 큰 값들의 10~12월 구매 비율
0.02393211289092296 

1.0보다 큰 값들의 10~12월 구매 비율
0.024507283633247643 

1.5보다 큰 값들의 10~12월 구매 비율
0.02550978958485661 

2.0보다 큰 값들의 10~12월 구매 비율
0.02772679345491467 

2.5보다 큰 값들의 10~12월 구매 비율
0.04968728283530229 

3.0보다 큰 값들의 10~12월 구매 비율
0.18411214953271027 

3.5보다 큰 값들의 10~12월 구매 비율
0.1875 

4.0보다 큰 값들의 10~12월 구매 비율
0.18407960199004975 

4.5보다 큰 값들의 10~12월 구매 비율
0.2727272727272727 



<ipython-input-2-a66e3072b375>:91: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  today = pd.datetime(year = 2022, month = 1, day = 1)


0